In [30]:
# mastermind
# https://github.com/analyticswithadam/Python/blob/main/Pull_all_Comments_and_Replies_for_YouTube_Playlists.ipynb

SyntaxError: invalid syntax (3812318127.py, line 2)

In [4]:
from googleapiclient.discovery import build
import pandas as pd
import getpass

In [11]:
api_key = getpass.getpass('Please enter your YouTube API key: ')
playlist_ids = ['PL3-OIwNPoC3JFEvZP_nSMPrib8viH2KX-']

Please enter your YouTube API key:  ········


In [12]:
# Build the YouTube client
youtube = build('youtube', 'v3', developerKey=api_key)

In [13]:
# This is the function to pull all comments from a youtube playlist. 
# Only include the text following the signin the playlists url.


def get_all_video_ids_from_playlists(youtube, playlist_ids):
    all_videos = []  # Initialize a single list to hold all video IDs

    for playlist_id in playlist_ids:
        next_page_token = None

        # Fetch videos from the current playlist
        while True:
            playlist_request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId=playlist_id,
                maxResults=50,
                pageToken=next_page_token)
            playlist_response = playlist_request.execute()

            all_videos += [item['contentDetails']['videoId'] for item in playlist_response['items']]

            next_page_token = playlist_response.get('nextPageToken')

            if next_page_token is None:
                break

    return all_videos

In [14]:
# GET ALL COMMENTS!!!

# Fetch all video IDs from the specified playlists
video_ids = get_all_video_ids_from_playlists(youtube, playlist_ids)

# Function to get replies for a specific comment
def get_replies(youtube, parent_id, video_id):  # Added video_id as an argument
    replies = []
    next_page_token = None

    while True:
        reply_request = youtube.comments().list(
            part="snippet",
            parentId=parent_id,
            textFormat="plainText",
            maxResults=100,
            pageToken=next_page_token
        )
        reply_response = reply_request.execute()

        for item in reply_response['items']:
            comment = item['snippet']
            replies.append({
                'Timestamp': comment['publishedAt'],
                'Username': comment['authorDisplayName'],
                'VideoID': video_id,
                'Comment': comment['textDisplay'],
                'Date': comment['updatedAt'] if 'updatedAt' in comment else comment['publishedAt']
            })

        next_page_token = reply_response.get('nextPageToken')
        if not next_page_token:
            break

    return replies

# Function to get all comments (including replies) for a single video
def get_comments_for_video(youtube, video_id):
    all_comments = []
    next_page_token = None

    while True:
        comment_request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            pageToken=next_page_token,
            textFormat="plainText",
            maxResults=100
        )
        comment_response = comment_request.execute()

        for item in comment_response['items']:
            top_comment = item['snippet']['topLevelComment']['snippet']
            all_comments.append({
                'Timestamp': top_comment['publishedAt'],
                'Username': top_comment['authorDisplayName'],
                'VideoID': video_id,  # Directly using video_id from function parameter
                'Comment': top_comment['textDisplay'],
                'Date': top_comment['updatedAt'] if 'updatedAt' in top_comment else top_comment['publishedAt']
            })

            # # Fetch replies if there are any
            # if item['snippet']['totalReplyCount'] > 0:
            #     all_comments.extend(get_replies(youtube, item['snippet']['topLevelComment']['id'], video_id))

        next_page_token = comment_response.get('nextPageToken')
        if not next_page_token:
            break

    return all_comments

# List to hold all comments from all videos
all_comments = []


for video_id in video_ids:
    video_comments = get_comments_for_video(youtube, video_id)
    all_comments.extend(video_comments)

# Create DataFrame
comments_df = pd.DataFrame(all_comments)

In [15]:
comments_df.to_csv('Rep_debate_comments.csv', index=False)
# # TO APPEND TO CSV:
# # new_df.to_csv('existing_data.csv', mode='a', index=False, header=False)

In [6]:
df = pd.read_csv('Rep_debate_comments.csv')

In [7]:
df.shape

(11005, 5)

In [8]:
df2 = pd.read_csv('Dem_debate_comments.csv')

In [9]:
df2.shape

(94219, 5)

In [ ]:
# WORKED GREAT FOR COMMENTS FROM ONE VIDEO


def getcomments(video):
  request = youtube.commentThreads().list(
      part="snippet",
      videoId=video,
      maxResults=100
  )

  comments = []

  # Execute the request.
  response = request.execute()

  # Get the comments from the response.
  for item in response['items']:
      comment = item['snippet']['topLevelComment']['snippet']
      public = item['snippet']['isPublic']
      comments.append([
          comment['authorDisplayName'],
          comment['publishedAt'],
          comment['likeCount'],
          comment['textOriginal'],
          comment['videoId'],
          public
      ])

  while (1 == 1):
    try:
     nextPageToken = response['nextPageToken']
    except KeyError:
     break
    nextPageToken = response['nextPageToken']
    # Create a new request object with the next page token.
    nextRequest = youtube.commentThreads().list(part="snippet", videoId=video, maxResults=100, pageToken=nextPageToken)
    # Execute the next request.
    response = nextRequest.execute()
    # Get the comments from the next response.
    for item in response['items']:
      comment = item['snippet']['topLevelComment']['snippet']
      public = item['snippet']['isPublic']
      comments.append([
          comment['authorDisplayName'],
          comment['publishedAt'],
          comment['likeCount'],
          comment['textOriginal'],
          comment['videoId'],
          public
      ])

  df2 = pd.DataFrame(comments, columns=['author', 'updated_at', 'like_count', 'text','video_id','public'])
  return df2

In [10]:
# request = youtube.commentThreads().list(
#     part="snippet",
#     videoId="ltNVyvK8Paw",
#     maxResults=1000
# )
# response = request.execute()

# comments = []

# for item in response['items']:
#     comment = item['snippet']['topLevelComment']['snippet']
#     comments.append([
#         comment['authorDisplayName'],
#         comment['publishedAt'],
#         comment['updatedAt'],
#         comment['likeCount'],
#         comment['textDisplay']
#     ])


# # TO APPEND TO CSV:
# # new_df.to_csv('existing_data.csv', mode='a', index=False, header=False)
# df = pd.DataFrame(comments, columns=['author', 'published_at', 'updated_at', 'like_count', 'text'])

# df.head(10)

,author,published_at,updated_at,like_count,text
0,@michelecruz6365,2024-12-07T16:51:50Z,2024-12-07T16:51:50Z,1,The one thing i so admire about President Trum...
1,@MyLifeTheStruggle1,2024-11-10T03:41:11Z,2024-11-10T03:41:11Z,0,2024 304
2,@alfredm5400,2024-11-03T23:23:39Z,2024-11-03T23:23:39Z,0,trump 2024 🎉
3,@MarcosAntonio-z8y9h,2024-10-22T20:48:18Z,2024-10-22T20:48:18Z,0,@Buenas noches fuerza brutalidad hoy coneza el...
4,@langtran-nn2jq,2024-10-22T20:43:31Z,2024-10-22T20:43:31Z,0,❤❤❤❤❤❤
5,@MarcosAntonio-z8y9h,2024-10-22T20:40:52Z,2024-10-22T20:40:52Z,0,@Thank you God for forgive Ness though the pre...
6,@mdjahangirhossain-l2d,2024-10-22T09:06:05Z,2024-10-22T09:06:05Z,0,"Well done ❤❤❤❤❤ <a href=""https://www.youtube.c..."
7,@marienzasokau8127,2024-10-22T08:57:40Z,2024-10-22T08:57:40Z,0,"It is now October, the twenty second day of th..."
8,@Juana.V.V,2024-10-16T09:25:09Z,2024-10-16T09:25:09Z,0,u s e presidente republicano presiden...
9,@Eugenio-c3o,2024-10-14T10:05:58Z,2024-10-14T10:05:58Z,0,I&#39;M PALAUAN FROM PALAU ISLAND LOCATED IN W...


In [9]:
# new_df.to_csv('existing_data.csv', mode='a', index=False, header=False)


df.to_csv('trump_comments.csv', index=False)

In [ ]:
# # Export whole dataset to the local machine as CSV File
# csv_file = 'comments_data.csv'  # Name your file
# comments_df.to_csv(csv_file, index=False)

# from google.colab import files

# # Trigger a download to your local machine
# files.download(csv_file)

In [ ]:
# # Function to get replies for a specific comment
# def get_replies(youtube, parent_id, video_id):  # Added video_id as an argument
#     replies = []
#     next_page_token = None

#     while True:
#         reply_request = youtube.comments().list(
#             part="snippet",
#             parentId=parent_id,
#             textFormat="plainText",
#             maxResults=100,
#             pageToken=next_page_token
#         )
#         reply_response = reply_request.execute()

#         for item in reply_response['items']:
#             comment = item['snippet']
#             replies.append({
#                 'Timestamp': comment['publishedAt'],
#                 'Username': comment['authorDisplayName'],
#                 'VideoID': video_id,
#                 'Comment': comment['textDisplay'],
#                 'Date': comment['updatedAt'] if 'updatedAt' in comment else comment['publishedAt']
#             })

#         next_page_token = reply_response.get('nextPageToken')
#         if not next_page_token:
#             break

#     return replies

In [ ]:
# # Function to get all comments (including replies) for a single video
# def get_comments_for_video(youtube, video_id):
#     all_comments = []
#     next_page_token = None

#     while True:
#         comment_request = youtube.commentThreads().list(
#             part="snippet",
#             videoId=video_id,
#             pageToken=next_page_token,
#             textFormat="plainText",
#             maxResults=100
#         )
#         comment_response = comment_request.execute()

#         for item in comment_response['items']:
#             top_comment = item['snippet']['topLevelComment']['snippet']
#             all_comments.append({
#                 'Timestamp': top_comment['publishedAt'],
#                 'Username': top_comment['authorDisplayName'],
#                 'VideoID': video_id,  # Directly using video_id from function parameter
#                 'Comment': top_comment['textDisplay'],
#                 'Date': top_comment['updatedAt'] if 'updatedAt' in top_comment else top_comment['publishedAt']
#             })

#             # Fetch replies if there are any
#             if item['snippet']['totalReplyCount'] > 0:
#                 all_comments.extend(get_replies(youtube, item['snippet']['topLevelComment']['id'], video_id))

#         next_page_token = comment_response.get('nextPageToken')
#         if not next_page_token:
#             break

#     return all_comments
